# Solving the 1-D Diffusion Equation (Implicit)

Let us now apply the implicit finite difference scheme to our 1-D heat diffusion problem. To recap, here is the problem once again:

## The Problem

We are interested in how the temperature $T$ changes with time $t$, given a specified initial temperature $T_{0}[x,t_{0}]$ and boundary conditions; e.g., a dike intrusion in the lithosphere ($L$ = 100 m; $W$ = 5 m; $\kappa$ = 10⁻⁶ m²/s).

<img src="../Figures/Exercise02_1.png" alt="drawing" width="350"/> <br>
Fig. 1. Sketch of the geological problem and the profile of the initial temperature condition.

How long would it take for the dike to cool to a specified temperature?

To determine the cooling duration, we must solve the temperature diffusion equation (a parabolic ODE):

$$
\rho c_p \frac{\partial{T}}{\partial{t}} = \frac{\partial{}}{\partial{x}} \left( k\frac{\partial{T}}{\partial{x}} \right), \tag{1}
$$

where $k$ is the thermal conductivity [W/m/K], $T$ the temperature [K], and $\rho$ the density [kg/m³]. By rearranging (assuming the thermal parameters are constant!), we can express the equation using the thermal diffusivity $\kappa = \frac{k}{\rho c_p}$:

$$
\frac{\partial{T}}{\partial{t}} = \kappa \frac{\partial^2{T}}{\partial{x^2}}. \tag{2}
$$

### Finite Difference Approximation

To solve the problem numerically using finite differences, we first need to construct a numerical grid (the discretization):

<img src="../Figures/Exercise02_2.png" alt="drawing" width="600"/> <br>
Fig. 2. Numerical 1-D grid for discretizing the ODE.

To solve the diffusion equation numerically using the implicit finite difference scheme, we reformulate the PDE as follows:

$$
\frac{T_{i}^{n+1}-T_{i}^{n}}{\Delta t} = \frac{T_{i+1}^{n+1} - 2T_{i}^{n+1} + T_{i-1}^{n+1}}{(\Delta x)^2} \tag{3}
$$

This equation can be rewritten to form a system of equations, with as many equations as interior grid points:

$$
-a T_{i-1}^{n+1} + \left(2a + b\right) T_{i}^{n+1} - a T_{i+1}^{n+1} = b T_{i}^{n}, \tag{4}
$$

with 

$$
a=\frac{\kappa}{\Delta{x^2}}, b = \frac{1}{\Delta{t}}. \tag{5}
$$

Thus, we obtain a tridiagonal system of equations, which can be represented by a coefficient matrix $\mathbf{A}$, an unknown vector $T^{,n+1}$, and a known vector $T^n$.

Let us first load the necessary modules for the numerical solution of our problem. We will begin by programming the solvers ourselves, before using the predefined solvers in GeoModBox.jl. The predefined solver is located in the submodule GeoModBox.HeatEquation.OneD.

In [ ]:
using Plots, ExtendableSparse, LinearAlgebra, Printf
using GeoModBox.HeatEquation.OneD
start = time()

### Parameterdefinition

Let us first define the parameters for the problem (physical constants, numerical domain, and time parameters):

In [ ]:
alternative =   1
# Physics --------------------------------------------------------------- #
L           =   100.0   # Length [m]
Tdike       =   1200.0  # Dike Temperature [C]
Trock       =   300.0   # Backgraound Temperature [C]
κ           =   1.0e-6  # Thermal Diffusivity [m2/s]
W           =   5.0     # Dike width [m]
# ----------------------------------------------------------------------- #
# Numerical Parameter --------------------------------------------------- #
nc          =   100                 # Number of centroid
Δx          =   L/nc                # Grid spacing
xc          =   Δx/2:Δx:(L-Δx/2)    # x-coordinates
# Iterations
niter       =   10  
ϵ           =   1.0e-10       
# ----------------------------------------------------------------------- #
# Time Parameter -------------------------------------------------------- #
day         =   3600.0*24.0     # Seconds per day
fac         =   1.0 
Δt          =   fac*Δx^2.0 / (2*κ)
tmax        =   365.0*day 
nt          =   ceil(Int,tmax/Δt)
Time        =   0
# ----------------------------------------------------------------------- #

### Initial Conditions

To solve our problem, we first need to define the initial conditions. We assume 300 °C for the host rock and 1200 °C for the dike. That is, the initial temperature is defined by:

$$
T \left(x < \left( \frac{L}{2} - W \right), x > \left( \frac{L}{2} + W \right), t = 0 \right) = 300, \tag{6}
$$
$$
T \left(x > \left( \frac{L}{2} - W \right), x < \left( \frac{L}{2} + W \right), t = 0 \right) = 1200. \tag{7}
$$

Next, we will visualize the initial condition graphically.

In [ ]:
# Initial Condition; Temperature profile -------------------------------- #
T       =   (
        T       =    zeros(nc), 
        T0      =    zeros(nc), 
        T_ex    =    zeros(nc+2), 
        R       =    zeros(nc),
        ∂T2∂x2  =    zeros(nc))
T.T    .= Trock                         # Background T
@. T.T[abs(xc-L/2) <= W/2] =  Tdike     # Dike T
T.T_ex[2:end-1]         .=      T.T
T.T0                    .=      T.T
# ----------------------------------------------------------------------- #
# Plot initial condition ------------------------------------------------ #
p = plot(xc, T.T, xlabel="x [m]", ylabel="T [°C]", 
        title="Temperature distribution after 
        $(round(Time/day, digits=1)) days", 
        ylim=(0, Tdike))
display(p)
# ----------------------------------------------------------------------- #

### Boundary Conditions

Since we use central grid points for the temperature, no grid point lies directly on the boundaries (which, contrary to expectation, is actually an advantage!). To impose temperature boundary conditions, we therefore make use of additional ghost nodes (see Figure 2). That is, we determine the temperature at the ghost nodes in order to solve the temperature partial differential equation at the nearest interior grid point using finite differences.

For a constant temperature condition at the boundaries (Dirichlet), we can determine the temperature at the ghost nodes by linear interpolation, so that:

**West**
$$
T_{Ghost,W} = 2 T_{BC,W} + T_{1}, \tag{8}
$$
**East**
$$
T_{Ghost,E} = 2 T_{BC,E} + T_{nc}. \tag{9}
$$

For constant flux boundary conditions, the temperature at the boundaries is given by:

**West**
$$
T_{Ghost,W} = T_{1} - c_W \Delta x, \tag{10}
$$
**East**
$$
T_{Ghost,E} = T_{1} + c_E \Delta x, \tag{11}
$$

where $c_W = \frac{\partial T}{\partial x}$ and $c_E = \frac{\partial T}{\partial x}$ define the flux conditions at the respective boundaries.


Since we have a tridiagonal system of equations, the coefficients and the right-hand side for the equations of the first and last interior grid points must be modified depending on the boundary conditions as follows (derivation see lecture):

#### **Dirichlet**
*West*
$$
\left(3 a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T{1}^{n} + 2 a T_{BC,W} \tag{12}
$$
*East*
$$
- a T_{nc-1}^{n+1} + \left(3 a + b \right) T_{nc}^{n+1} = b T_{nc}^{n} + 2 a T_{BC,E} \tag{13}
$$

#### **Neumann**
*West*
$$
\left(a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T_{1}^n - a c_{W} \Delta{x} \tag{14}
$$
*East*
$$
- a T_{nc-1}^{n+1} + \left(a + b \right) T_{nc}^{n+1}  = b T_{nc}^n - a c_{E} \Delta{x} \tag{15}
$$

When initializing the boundary conditions in the code, we use a trick and define the temperature at the ghost nodes later in the script.

In [ ]:
# Boundary Conditions --------------------------------------------------- #
BC   = (
    type = (W=:Dirichlet, E=:Dirichlet),
    # type = (W=:Neumann, E=:Neumann),
    val  = (W=300.0, E=300.0))
#end
# ----------------------------------------------------------------------- #

### System of Equations

Let us now initialize the coefficient matrix and the right-hand side:

In [ ]:
# Assemble Coefficient Matrix ------------------------------------------- #
# Definition of the Matrix (in sparse form)
ndof        =   length(T.T)
K           =   ExtendableSparseMatrix(ndof,ndof)
rhs         =   zeros(nc)
# ----------------------------------------------------------------------- #

### Visualization

To visualize the results as an animation in a GIF file, we first need to specify the location and the name of the file:

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("03_1D_implicit_",alternative)
save_fig    =   1
# ----------------------------------------------------------------------- #          

### Time Loop

Numerically, we can now solve the PDE in a time loop in different ways (Alternative I is sufficient; those interested can also implement Alternatives II and III).

In [ ]:
# Timestep loop --------------------------------------------------------- #
for n = 1:nt
    println("Time Step: ",n,", Time: $(round(Time/day, digits=1)) [d]")
    if alternative == 1
        a   =   κ / Δx^2
        b   =   1 / Δt

        @. rhs     =   b * T.T

        # Alternative I
        for i = 1:nc  
            # Equation number
            ii          =   i
            # Stencil 
            iW          =   ii - 1
            iC          =   ii
            iE          =   ii + 1   
            # Boundaries 
            inW    =  i==1    ? false  : true
            DirW   = (i==1    && BC.type.W==:Dirichlet) ? 1. : 0.
            NeuW   = (i==1    && BC.type.W==:Neumann  ) ? 1. : 0.
            inE    =  i==nc ? false  : true
            DirE   = (i==nc && BC.type.E==:Dirichlet) ? 1. : 0.
            NeuE   = (i==nc && BC.type.E==:Neumann  ) ? 1. : 0.
            if inE
                K[ii,iE]    = - a
            end
            K[ii,iC]        =   (2 + DirW + DirE - NeuW - NeuE)*a + b
            if inW 
                K[ii,iW]    = - a
            end                            
            # Change right hand side due to boundary conditions ---
            rhs[i]  +=  2*a*BC.val.W * DirW - 
                            a*BC.val.W*Δx * NeuW + 
                            2*a*BC.val.E * DirE + 
                            a*BC.val.E*Δx * NeuE
        end            
        T.T     .=   K \ rhs
    elseif alternative == 2            
        BackwardEuler1Dc!(T, κ, Δx, Δt, nc, BC , K, rhs)
    elseif alternative == 3
        for iter = 1:niter
            # Residual iteration
            ComputeResiduals1Dc!(T, κ, Δx, Δt, BC )
            @printf("||R|| = %1.4e\n", norm(T.R)/length(T.R))            
            norm(T.R)/length(T.R) < ϵ ? break : nothing
            # Assemble linear system
            AssembleMatrix1Dc!( κ, Δx, Δt, nc, BC, K )
            # Solve for temperature correction: Cholesky factorisation
            Kc = cholesky(K.cscmatrix)
            # Solve for temperature correction: Back substitutions
            δT = -(Kc\T.R[:])                   
            # Update temperature            
            T.T .= T.T .+ δT            
        end        
        # Update temperature
        @. T.T0     =   T.T  
    end    
    # Calculate time ---
    Time    =   Time + Δt        
    # Plot Solution ---
    p = plot(xc, T.T, xlabel="x [m]", ylabel="T [°C]", 
            title="Temperature distribution after
            $(round(Time/day, digits=1)) days", 
            xlim=(0,L),ylim=(0, Tdike))
    if save_fig == 1
        Plots.frame(anim)
    else
        display(p)
    end    
end

We now need to create and save the animation:

In [ ]:
# Speicher Animation ---------------------------------------------------- #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
else
    display(p)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
# ----------------------------------------------------------------------- #
stop = time()
println(stop-start)